In [1]:
import os, subprocess, random, cv2, numpy
from PIL import Image
from io import BytesIO
from time import sleep
from pathlib import Path
from waiting import wait, TimeoutExpired

# You must have the adb executable in the same directory as this notebook!

'''Basic Functions'''

def start(emu):
    subprocess.run("./adb kill-server",shell=True)
    if emu == 'nox': subprocess.run("./adb connect 127.0.0.1:62001",shell=True)
    else: subprocess.run("./adb devices",shell=True)

def close():
    subprocess.run("./adb kill-server",shell=True)
    
def tap(x,y):
    subprocess.run("./adb shell input tap {x} {y}".format(x=x,y=y),shell=True)

def swipe(start,end,dur):
    # start = a location tuple
    # end = a location tuple
    # dur = how long in ms it takes to execute the swipe in seconds
    subprocess.run("./adb shell input swipe {x1} {y1} {x2} {y2} {dur}"
                   .format(x1=start[0],y1=start[1],x2=end[0],y2=end[1],dur=dur*1000),shell=True)

In [2]:
'''Derived functions'''

def rng_tap(x_tuple,y_tuple):
    # each tuple is a range: (low,high)
    tap(random.randint(x_tuple[0],x_tuple[1]),random.randint(y_tuple[0],y_tuple[1]))
    
def coord_tap(coord):
    # coord = a dictionary with a range for height and width: {'h':(low,high),'w':(low,high),'sleep':time in sec}
    tap(random.randint(coord['w'][0],coord['w'][1]),random.randint(coord['h'][0],coord['h'][1]))
    sleep(coord['sleep'])
    
def screenshot():
    raw = subprocess.run("./adb exec-out screencap -p",shell=True,stdout=subprocess.PIPE).stdout
    data = BytesIO(raw)
    im = Image.open(data).convert('RGB')
    im.save("screenshot.png")

def pixel_color(loc):
    # loc = the coordinates of the pixel you want to check
    raw = subprocess.run("./adb exec-out screencap -p",shell=True,stdout=subprocess.PIPE).stdout
    data = BytesIO(raw)
    return Image.open(data).convert('RGB').getpixel(loc)

def color_match(data):
    # data['loc'] = the coordinates of the pixel you want to check
    # data['two'] = the color tuple you are checking against
    loc,color = data['loc'],data['color']
    if len([n for n,v in enumerate(pixel_color(loc)) if v-20<color[n]<v+20]) == 3: return True
    else: return False

def img_match(capture,template):
    # capture = a screenshot
    # template = the image you are looking for
    converted = cv2.cvtColor(numpy.array(capture), cv2.COLOR_RGB2GRAY)
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(template,None)
    kp2, des2 = sift.detectAndCompute(converted,None)
    matches = cv2.BFMatcher().knnMatch(des1,des2,k=2)
    good = [m for m,n in matches if m.distance < 0.6*n.distance] # 0.75 originally
    dst_pt = [kp2[m.trainIdx].pt for m in good]
    return tuple([numpy.mean(x) for x in zip(*dst_pt)])

In [3]:
'''Another Eden Functions'''

# Button Locations (1280x720)
unit = {'h':(600,715),'1':(20,175),'2':(190,345),'3':(360,510),'4':(530,685),'5':(725,875),'6':(895,1045)}
skill = {'h':(525,570),'1':(40,250),'2':(290,495),'3':(535,745),'4':(780,990),'swap':(1030,1235)}
atk_button = {'h':(560,670),'w':(1115,1225),'sleep':0.1}
af_button = {'h':(25,100),'w':(1175,1230),'sleep':0.1}
map_button = {'h':(630,685),'w':(250,310),'sleep':1.5}
unknown_era = {'h':(75,120),'w':(420,475),'sleep':0.2}
rift_loc = {'h':(380,410),'w':(450,700),'sleep':0.2}
move_confirm = {'h':(375,430),'w':(675,885),'sleep':3}
fruit_button = {'h':(40,85),'w':(410,455),'sleep':0.5}
clear_fruit = {'h':(100,550),'w':(100,1150),'sleep':1}
eat_confirm = {'h':(375,430),'w':(675,885),'sleep':2.2}

# Color Match: Locations + Reference Colors
battle_start = {'loc':(1209,498),'color':(231,221,163)}
victory_screen = {'loc':(66,105),'color':(255,250,183)}

def sequencer(data):
    # data = tuple/list of coordinate dictionaries
    for seq in data:
        coord_tap(seq)

def alt_double_swipe(n,dur=4):
    # n = an integer
    # dur = the swipe time in seconds
    if n%2 == 0: loc = (400,800)
    else: loc = (800,400)
    y = random.randint(300,500)
    swipe((loc[0],y),(loc[1],y),dur)
    if color_match(battle_start): return '' # This will skip the second swipe if a battle has started
    swipe((loc[1],y),(loc[0],y),dur)

def any_tap(time=0.1):
    # time = sleep time after tap in seconds
    tap(random.randint(100,1150),random.randint(100,550))
    sleep(time)

def swap(front_num,back_num=6):
    # front_num = the position number for the unit being swapped out (1-4)
    # back_num = the postion number for the unit being swapped in (5-6)
    if int(front_num) not in range(1,5): return 'wrong front unit number'
    if int(back_num) not in [5,6]: return 'wrong back unit number'
    rng_tap(unit[str(front_num)],unit['h'])
    rng_tap(skill['swap'],skill['h'])
    rng_tap(unit[str(back_num)],unit['h'])
    sleep(0.1)

def set_skill(unit_num,skill_num):
    # unit_num = the position number for the unit being swapped out (1-4)
    # skill_num = the position number for the skill being picked (1-4)
    if int(unit_num) not in range(1,5): return 'wrong unit number'
    if int(skill_num) not in range(1,5): return 'wrong skill number'
    rng_tap(unit[str(unit_num)],unit['h'])
    rng_tap(skill[str(skill_num)],skill['h'])
    sleep(0.1)

def heal():
    sequencer((map_button,unknown_era,rift_loc,move_confirm,fruit_button,clear_fruit,eat_confirm))
    any_tap() # Clear final notifications
    any_tap() # Clear final notifications
    any_tap(0.5) # Clear final notifications
    #coord_tap(map_button) # this can be done here or at the start of the relocation function

In [4]:
'''Another Eden Loops'''

def skill_spam(num,time=0.3):
    x,y = random.randint(skill[str(num)][0],skill[str(num)][1]),random.randint(skill['h'][0],skill['h'][1])
    while True:
        tap(x,y)
        sleep(time)

def continuous_tap(time=0.3):
    # picks a random loc near the center of the screen and taps there until stopped
    # time = the sleep time in seconds between taps
    x,y = random.randint(400,800),random.randint(200,400)
    while True:
        tap(x,y)
        sleep(time)
        
def af_spam(num=4):
    coord_tap(af_button)
    skill_spam(num)

def vc_farm():
    # 0MP farming loop using VC
    n,m = 0,0
    while True:
        n += 1
        print('moving',n)
        alt_double_swipe(n,6)
        try: wait(lambda:color_match(battle_start),timeout_seconds=6) # check for battle
        except TimeoutExpired: continue # if no battle restart loop to move again
        m += 1
        print('battle',m)
        swap(4,6)
        coord_tap(atk_button)
        sleep(5) # wait for victory screen to appear
        any_tap() # clear victory screen
        sleep(1) # wait for victory screen to clear before looping again

In [5]:
emu = 'bluestacks' # options are 'nox' and 'bluestacks'
start(emu)

In [6]:
#vc_farm()

In [16]:
continuous_tap()

KeyboardInterrupt: 